In [110]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import pandas as pd
from joblib import dump

from src.scripts.artemis_data import get_ns_cols, get_email_cols

In [120]:
file = "/Users/liam/PycharmProjects/dataspell/artemis/data/processed/final_whois_data.csv"
df = pd.read_csv(file)


In [121]:
label = 'malicious'
labels = df[label].values
data_df = df.drop(columns=label)

In [122]:
data_df = data_df.drop(columns=get_ns_cols(7))
data_df = data_df.drop(columns=get_email_cols(3))
data_df = data_df.drop(columns=['domain', 'updated_date', 'expiration_date', 'creation_date', 'days_since_creation'])

In [123]:
country_cols = ['country', 'dns_rec_a_cc', 'dns_rec_mx_cc']
countries_list = []
for col in country_cols:
    countries_list += data_df[col].tolist()
country_encoder = LabelEncoder().fit(countries_list)
for col in country_cols:
    data_df[col] = country_encoder.transform(data_df[col])

In [124]:
cols_requiring_encoding = [
    'registrar', 'dnssec', 'org', 'state', 'whois_server', 'address', 'city', 'name',
    'zipcode', 'dns_rec_a_org', 'dns_rec_mx_org', 'main_name_server_domain', 'email_domains',
]
encoder_dict = {}
for col in cols_requiring_encoding:
    enc = LabelEncoder().fit(data_df[col])
    encoder_dict[col] = enc
    data_df[col] = enc.transform(data_df[col])


In [125]:
data_df = data_df.fillna(-1)

In [127]:
X_train, X_test, y_train, y_test = train_test_split(data_df, labels, random_state=0)

In [128]:
clf = RandomForestClassifier(criterion='entropy', random_state=0).fit(X_train.values, y_train)

In [129]:
clf.score(X_test, y_test)

/Users/liam/miniconda3/envs/artemis/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.9687943262411347

In [130]:
model_loc = "../models/"
model_name = "rfc.joblib"
encoder_name = "enc_dict.joblib"
country_encoder_name = "country_encoder.joblib"

In [131]:
dump(clf, model_loc+model_name)

['../models/rfc.joblib']

In [132]:
dump(labelenc, model_loc+enc_name)

NameError: name 'labelenc' is not defined

In [133]:
dump(country_encoder, model_loc+country_encoder_name)

['../models/country_encoder.joblib']

In [134]:
clf.feature_importances_

array([1.17393549e-02, 2.54516430e-05, 4.64005626e-02, 1.85798990e-02,
       4.51685819e-03, 3.44429112e-02, 2.66360212e-02, 4.01854745e-02,
       1.11457694e-02, 1.88012447e-02, 1.50510036e-02, 1.69043764e-02,
       2.91366194e-02, 2.26840007e-02, 5.36916017e-02, 3.75048480e-02,
       9.14764214e-02, 2.22214178e-02, 1.91158100e-02, 9.56943277e-02,
       5.27881170e-02, 3.73292456e-02, 4.26937981e-03, 1.40687642e-02,
       5.08221293e-02, 5.10247703e-02, 4.48341454e-03, 3.32597543e-03,
       1.18041956e-02, 3.40587823e-03, 4.03912632e-02, 9.97214025e-05,
       4.56686246e-03, 1.95740167e-05, 3.07150723e-03, 2.18224484e-02,
       7.55484480e-03, 6.85588584e-02, 4.63907624e-03])

In [140]:
data_df[data_df.columns.tolist()[19]]

0       3.78212
1       3.87314
2       3.61437
3       3.87314
4       3.88018
         ...   
5633    4.51378
5634    4.51378
5635    3.91613
5636    4.02895
5637    3.70282
Name: max_entropy, Length: 5638, dtype: float64